In [183]:
import numpy as np, pandas as pd
import ast 
from sklearn import linear_model
from sklearn import metrics
from sklearn.cross_validation import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [184]:
data = pd.read_csv("train_detect_sent.csv")

In [185]:
data.head(3)

,answer_start,context,question,text,sentences,target,sent_emb,quest_emb,cosine_sim,euclidean_dis,pred_idx_cos,pred_idx_euc
0,515,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous,"['Architecturally, the school has a Catholic c...",5,"[array([ 0.05519997, 0.05013141, 0.04787038,...",[[ 0.1095634 0.11422941 0.04428943 ... 0.0...,"[0.43262487532502947, 0.36864841025319484, 0.3...","[14.108409, 15.0115595, 18.043663, 13.160972, ...",5,5
1,188,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,a copper statue of Christ,"['Architecturally, the school has a Catholic c...",2,"[array([ 0.05519997, 0.05013141, 0.04787038,...",[[ 0.10951651 0.11030623 0.05210007 ... -0.0...,"[0.45407456884452513, 0.32262004808444933, 0.3...","[12.889506, 12.285219, 16.843704, 8.361172, 11...",3,3
2,279,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,the Main Building,"['Architecturally, the school has a Catholic c...",3,"[array([ 0.05519997, 0.05013141, 0.04787038,...",[[ 0.00262268 0.14930707 0.03063984 ... 0.0...,"[0.39596294272736, 0.3157232094505895, 0.34619...","[11.590836, 12.180486, 16.56459, 7.4446793, 9....",3,3


In [186]:
ast.literal_eval(data["sentences"][0])

['Architecturally, the school has a Catholic character.',
 "Atop the Main Building's gold dome is a golden statue of the Virgin Mary.",
 'Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".',
 'Next to the Main Building is the Basilica of the Sacred Heart.',
 'Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.',
 'It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.',
 'At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.']

In [187]:
data = data[data["sentences"].apply(lambda x: len(ast.literal_eval(x)))<11].reset_index(drop=True)

In [188]:
def create_features(data):
    train = pd.DataFrame()
     
    for k in range(len(data["euclidean_dis"])):
        dis = ast.literal_eval(data["euclidean_dis"][k])
        for i in range(len(dis)):
            train.loc[k, "column_euc_"+"%s"%i] = dis[i]
    
    print("Finished")
    
    for k in range(len(data["cosine_sim"])):
        dis = ast.literal_eval(data["cosine_sim"][k].replace("nan","10000"))
        for i in range(len(dis)):
            train.loc[k, "column_cos_"+"%s"%i] = dis[i]
            
    train["target"] = data["target"]
    return train

In [189]:
train = create_features(data)

Finished


In [190]:
train.head(3)

,column_euc_0,column_euc_1,column_euc_2,column_euc_3,column_euc_4,column_euc_5,column_euc_6,column_euc_7,column_euc_8,column_euc_9,...,column_cos_1,column_cos_2,column_cos_3,column_cos_4,column_cos_5,column_cos_6,column_cos_7,column_cos_8,column_cos_9,target
0,14.108409,15.011560,18.043663,13.160972,12.855492,9.627245,15.943851,NaN,NaN,NaN,...,0.368648,0.367699,0.379501,0.374212,0.193091,0.365788,NaN,NaN,NaN,5
1,12.889506,12.285219,16.843704,8.361172,11.918098,17.601221,14.929258,NaN,NaN,NaN,...,0.322620,0.355004,0.271561,0.392342,0.384383,0.362597,NaN,NaN,NaN,2
2,11.590836,12.180486,16.564590,7.444679,9.165415,14.745420,13.291000,NaN,NaN,NaN,...,0.315723,0.346192,0.235901,0.294102,0.313559,0.317138,NaN,NaN,NaN,3


In [191]:
train.fillna(10000, inplace=True)

In [192]:
train.head(3).transpose()

,0,1,2
column_euc_0,14.108409,12.889506,11.590836
column_euc_1,15.011560,12.285219,12.180486
column_euc_2,18.043663,16.843704,16.564590
column_euc_3,13.160972,8.361172,7.444679
column_euc_4,12.855492,11.918098,9.165415
column_euc_5,9.627245,17.601221,14.745420
column_euc_6,15.943851,14.929258,13.291000
column_euc_7,10000.000000,10000.000000,10000.000000
column_euc_8,10000.000000,10000.000000,10000.000000
column_euc_9,10000.000000,10000.000000,10000.000000


### Fitting Multinomial Logistic Regression

In [201]:
train_x, test_x, train_y, test_y = train_test_split(train.iloc[:,:-1],
train.iloc[:,-1], train_size=0.9, random_state = 5)

In [202]:
# Train multinomial logistic regression model
mul_lr = linear_model.LogisticRegression(multi_class='multinomial', solver='newton-cg', C=100)
mul_lr.fit(train_x, train_y)

print("Multinomial Logistic regression Train Accuracy : ", metrics.accuracy_score(train_y, mul_lr.predict(train_x)))
print("Multinomial Logistic regression Test Accuracy : ", metrics.accuracy_score(test_y, mul_lr.predict(test_x)))


Multinomial Logistic regression Train Accuracy :  0.42735042735042733
Multinomial Logistic regression Test Accuracy :  0.38461538461538464
